In [ ]:
import json
import pandas as pd

In [ ]:
!mv logs_LTAT.03.001_20210207-0935-AT_01.xlsx logs_LTAT.03.001_20210207-0935_AT_01.xlsx

df = pd.read_excel("logs_LTAT.03.001_20210207-0935_AT_01.xlsx", engine="openpyxl", dtype=object, header=0)

In [ ]:
df.head()

In [ ]:
all_students_names = set(df["Kasutaja täisnimi"])
all_students_names

In [ ]:
time_string = df["Aeg"][0]
time_string

In [ ]:
import time
import datetime
  
print(time.mktime(datetime.datetime.strptime(time_string, "%d.%m.%Y %H:%M:%S").timetuple()))

In [ ]:
def get_time(time_string: str) -> int:
    return int(time.mktime(datetime.datetime.strptime(time_string, "%d.%m.%Y %H:%M:%S").timetuple()))

In [ ]:
def get_time_interval(df) -> int:
    time_list = [get_time(the_string) for the_string in df["Aeg"]]
    return max(time_list) - min(time_list)

In [ ]:
time_interval = get_time_interval(df)
time_interval

In [ ]:
print(f"It's {time_interval // 3600} hours, or {time_interval // 86400} days, or {time_interval // (86400 * 7)} weeks")
print("Why? Because some users may access the system in whole year (teachers?)")

In [ ]:
df_score = pd.read_excel("02-05 Hinded.xlsx", engine="openpyxl", dtype=object, header=0)
all_scored_students = set()
for index, row in df_score.iterrows():
    full_name = f"{row['Eesnimi']} {row['Perenimi']}"
    all_scored_students.add(full_name)

In [ ]:
all_scored_students

In [ ]:
len(all_scored_students)

In [ ]:
all_students_names = all_students_names & all_scored_students
all_students_names

In [ ]:
def get_time_interval(df) -> int:
    time_list = [get_time(row["Aeg"]) for _, row in df.iterrows() if row["Kasutaja täisnimi"] in all_students_names]
    return max(time_list) - min(time_list)

In [ ]:
time_interval = get_time_interval(df)
print(f"It's {time_interval // 3600} hours, or {time_interval // 86400} days, or {time_interval // (86400 * 7)} weeks")
print(f"Still > 16 weeks, some students used the system in Jan or Feb")

In [ ]:
def get_time_min(df) -> int:
    time_list = [get_time(row["Aeg"]) for _, row in df.iterrows() if row["Kasutaja täisnimi"] in all_students_names]
    return min(time_list)
time_min = get_time_min(df)
time_max = time_min + 86400 * (7 * 17) # We will use this later

In [ ]:
# For simplicity sake, we only consider "Components" just for now
all_components = list(set(df["Komponent"]))
all_components.sort()
all_components

In [ ]:
def get_student_dict(df) -> dict:
    students = {}
    for _, row in df.iterrows():
        time = get_time(row['Aeg'])
        if time > time_max:
            continue
        name = row['Kasutaja täisnimi']
        if name not in all_students_names:
            continue
        if not students.get(name):
            students[name] = {}
        component = row['Komponent']
        if not students[name].get(component):
            students[name][component] = 1
        else:
            students[name][component] += 1
    return students

In [ ]:
students = get_student_dict(df)

In [ ]:
print(len(students))
students

In [ ]:
def get_data_dict(df, label) -> dict:
    students_dict = get_student_dict(df)
    new_dict = {}
    for name in students_dict:
        new_dict[name] = []
        for com in all_components:
            value = students_dict[name].get(com, 0) / 17
            new_dict[name].append(value)
        new_dict[name].append(label)
    return new_dict, list(students_dict)

In [ ]:
data_dict, names = get_data_dict(df, 1)
data_list = [data_dict[k] for k in data_dict]

In [ ]:
print(len(data_list))
data_list

In [ ]:
def get_score(df, names):
    scores = []
    the_dict = {}
    for _, row in df.iterrows():
        the_dict[f"{row['Eesnimi']} {row['Perenimi']}"] = row["Kogutulemus (Punktid)"]
    for name in names:
        score = the_dict.get(name)
        scores.append(score)
    return scores

scores = get_score(df_score, names)
scores

In [ ]:
from pandas.core.frame import DataFrame
data = DataFrame(data_list)
data.columns = all_components + ["label"]
data['y'] = scores

In [ ]:
data.tail()

In [ ]:
import os
os.listdir() 

In [ ]:
all_labels = set()

for file in os.listdir():
    if not file.startswith('logs_') or not file.endswith('.xlsx'):
        continue
    spc = file.split('-')[1].split('_')[1]
    all_labels.add(spc)

all_labels = list(all_labels)
all_labels_dict = {}

for i in range(len(all_labels)):
    all_labels_dict[all_labels[i]] = i + 1
    
all_labels_dict

In [ ]:
total_data_list = []
total_scores = []

for file in os.listdir():
    global label, df, all_students_names, time_min, time_max, data_dict, names, data_list

    if not file.startswith('logs_') or not file.endswith('.xlsx') or "veebirühm" in file:
        continue
        
    print(f"Processing {file}...")
    
    spc = file.split('-')[1].split('_')[1]
    label = all_labels_dict[spc]
    df = pd.read_excel(file, engine="openpyxl", dtype=object, header=0)
    all_students_names = set(df["Kasutaja täisnimi"])
    all_students_names = all_students_names & all_scored_students
    time_min = get_time_min(df)
    time_max = time_min + 86400 * (7 * 17)
    data_dict, names = get_data_dict(df, label)
    data_list = [data_dict[k] for k in data_dict]
    scores = get_score(df_score, names)
    total_data_list = total_data_list + data_list
    total_scores = total_scores + scores

print(len(total_data_list))
print(len(total_scores))

In [ ]:
total_data = DataFrame(total_data_list)
total_data.columns = all_components + ["label"]
total_data['y'] = total_scores

In [ ]:
total_data.head()

In [ ]:
total_data.tail()

In [ ]:
total_data.to_csv('first_try_data.csv', encoding='utf-8', index=False)